Install libraries, only needed for the first run

In [1]:
%pip install pyyaml openai==1.0.0

Note: you may need to restart the kernel to use updated packages.


Wrapped up methods to call openai

In [2]:
from yaml import safe_load
import openai
GPT_ENGINE = 'gpt-4'

In [3]:
with open("openai.credential", 'r') as stream:
        credential_data = safe_load(stream)
openai_config = credential_data['openai']
openai.api_type = "azure"
openai.azure_endpoint = openai_config['endpoint']
openai.api_version = "2023-03-15-preview"
openai.api_key = openai_config["key"]
print(openai.version.VERSION)

def call_api(messages):    
    return openai.chat.completions.create(
        model=GPT_ENGINE,
        messages = messages,
        max_tokens = 3000
    )

def call_with_context(context: list, prompt: str, role='user') -> str:
    context.append({'role': role, 'content': prompt})
    response = call_api(context)
    message = response.choices[0].message
    context.append({"role": message.role, "content": message.content})
    return message.content

1.0.0


The main prompt. This is current static, and we should test and see how to improve it. Also, the current prompt only do calassification. We need to think about how to fill in the "elements" under each category.

In [6]:
ORACLE = """\
Hi, can we do an exercise related to data privacy? Many websites have the "privacy policy", and a paper proposed that there are 10 categories of data practice in such privacy contents:

1. First Party Collection/Use: how and why a
service provider collects user information.
2. Third Party Sharing/Collection: how user information may be shared with or collected by
third parties.
3. User Choice/Control: choices and control
options available to users.
4. User Access, Edit, & Deletion: if and how
users may access, edit, or delete their information.
5. Data Retention: how long user information is
stored.
6. Data Security: how user information is protected.
7. Policy Change: if and how users will be informed about changes to the privacy policy.
8. Do Not Track: if and how Do Not Track signals3
for online tracking and advertising are
honored.
9. International & Specific Audiences: practices
that pertain only to a specific group of users
(e.g., children, Europeans, or California residents).
10. Other: additional sub-labels for introductory or general text, contact information, and
practices not covered by the other categories.

I have a specific task for you: The user will give you a sentence from some privacy contents, and you tell them which category of practices this sentence may contain. 
"""

context = [{"role":"system", "content":ORACLE}]

A first try with only one sentence. We can do this step by step:
1. manually create a list or dictionary with some sentences from the corpus
2. read the csv files from the corpus to extract some sentences and their annotations.

In [9]:
samples = [
    "By visiting Amazon.com, you are accepting the practices described in this Privacy Notice",
    "If you do not want to receive e-mail or other mail from us, please adjust your Customer Communication Preferences.",
    "When you download or use apps created by Amazon or our subsidiaries, we may receive information about your location and your mobile device, including a unique identifier for your device",
    "Information about our customers is an important part of our business, and we are not in the business of selling it to others. "
]

for sentence in samples:
    answer = call_with_context(context, sentence)
    print("Sentence: " + sentence)
    print("Answer: " + answer)
    print("---------------")

Sentence: By visiting Amazon.com, you are accepting the practices described in this Privacy Notice
Answer: This sentence falls under the category 10: Other. It serves as an introductory statement that informs users they are agreeing to the practices described in Amazon's Privacy Notice upon visiting the website.
---------------
Sentence: If you do not want to receive e-mail or other mail from us, please adjust your Customer Communication Preferences.
Answer: This sentence falls under the category 3: User Choice/Control. It informs users that they have the option to adjust their communication preferences if they do not wish to receive emails or other communication from the service provider.
---------------
Sentence: When you download or use apps created by Amazon or our subsidiaries, we may receive information about your location and your mobile device, including a unique identifier for your device
Answer: This sentence falls under the category 1: First Party Collection/Use. It explains